## Creates COCO subset
Creates a subset of the COCO dataset with the specified attributes.

In [ ]:
from pathlib import Path

import torch
from tqdm import tqdm

from ssdv2.dataset import DatasetManager
from ssdv2.structs import FrameLabels

In [ ]:
DATASET_DIR = Path("/mnt/data/datasets/object_detection/coco/")
KEEP_CLASS_IDS = [0, 2, 3, 5, 7]
DTYPE = torch.float32
DEVICE = torch.device("cpu")

In [ ]:
manager = DatasetManager(DATASET_DIR)

In [ ]:
train_label_files = list(manager.train_labels_dir.glob("*.txt"))
for label_file in tqdm(train_label_files):
    objects = FrameLabels.from_file(label_file, DTYPE, DEVICE)